# PROBLEMA 2

**Resolver a Lei Fundamental da Dinâmica (LFD) no caso de um objeto de massa m = 1,0kg, submetido às forças peso e de arraste (proporcional à velocidade), num problema de dimensão 1. Consideremos o coeficiente é: α = 1,0 N/(m/s). Condição inicial: v(t₀) = v₀. Intervalo de tempo considerado: t ∈ [t₀, t₁] com t₀ = 0s, t₁ = 5,0s, e g = 9,8m/s²**

A LFD conduz à seguinte equação:

$$m\frac{dv}{dt}(t) = mg - \alpha v(t)$$

A implementação do MDF conduz a uma relação iterativa que permite calcular a velocidade em função do tempo em pontos discretos do intervalo [t₀, t₁]:

$$v_{j+1} = \left(1 - \frac{\alpha}{m}\delta t\right)v_j + g\delta t$$


In [ ]:
# ETAPA 1: Definir parâmetros do problema
using Plots

m = 1.0      # massa (kg)
α = 1.0      # coeficiente de arraste (N/(m/s))
g = 9.8      # gravidade (m/s²)
t₀ = 0.0     # tempo inicial (s)
t₁ = 5.0     # tempo final (s)
v₀ = 0.0     # velocidade inicial (m/s)

In [ ]:
# ETAPA 2: Implementar método numérico (Diferenças Finitas)
# Fórmula iterativa: v_{j+1} = (1 - α/m * dt) * v_j + g * dt

function resolver_velocidade(dt)
    # Criar vetor de tempo
    t = t₀:dt:t₁
    n = length(t)
    v = zeros(n)
    v[1] = v₀
    
    # Calcular velocidade em cada passo
    for j in 1:(n-1)
        v[j+1] = (1 - α/m * dt) * v[j] + g * dt
    end
    
    return t, v
end

# Testar com diferentes passos de tempo
dt_values = [0.1, 0.05, 0.01]
resultados = [resolver_velocidade(dt) for dt in dt_values]

In [ ]:
# ETAPA 3: Calcular solução analítica (para comparação)
# v(t) = (mg/α) * (1 - e^(-αt/m)) + v₀ * e^(-αt/m)

function solucao_analitica(t)
    v_terminal = m * g / α
    return v_terminal * (1 - exp(-α * t / m)) + v₀ * exp(-α * t / m)
end

t_analitico = range(t₀, t₁, length=1000)
v_analitico = solucao_analitica.(t_analitico)
v_terminal = m * g / α  # Velocidade limite

In [ ]:
# ETAPA 4: Plotar resultados
p = plot(size=(800, 600))

# Soluções numéricas
cores = [:blue, :green, :red]
for (i, dt) in enumerate(dt_values)
    t, v = resultados[i]
    plot!(p, t, v, marker=:o, markersize=3, color=cores[i], 
          label="Numérico (dt=$dt s)")
end

# Solução analítica
plot!(p, t_analitico, v_analitico, linewidth=3, color=:black, 
      label="Analítico")

# Velocidade terminal
hline!(p, [v_terminal], linestyle=:dash, color=:orange, 
       label="v_terminal = $(v_terminal) m/s")

xlabel!(p, "Tempo (s)")
ylabel!(p, "Velocidade (m/s)")
title!(p, "Velocidade vs Tempo - Queda com Arraste")

In [ ]:
# ETAPA 5: Análise de erro
println("Velocidade em t = $t₁ s:")
println("Analítico: $(round(solucao_analitica(t₁), digits=4)) m/s\n")

for (i, dt) in enumerate(dt_values)
    t, v = resultados[i]
    erro = abs(v[end] - solucao_analitica(t₁))
    erro_percent = 100 * erro / solucao_analitica(t₁)
    println("dt = $dt s: $(round(v[end], digits=4)) m/s (erro: $(round(erro_percent, digits=3))%)")
end

## Resultados e Conclusões

### Equação Diferencial Resolvida

A equação fundamental da dinâmica para este problema é:

$$m\frac{dv}{dt} = mg - \alpha v$$

### Solução Analítica

A resolução analítica desta equação diferencial de primeira ordem conduz à expressão:

$$v(t) = \frac{mg}{\alpha}\left(1 - e^{-\frac{\alpha t}{m}}\right) + v_0 e^{-\frac{\alpha t}{m}}$$

### Velocidade Terminal

A velocidade limite (terminal) é obtida quando $\frac{dv}{dt} = 0$:

$$v_{limite} = \frac{mg}{\alpha} = \frac{1.0 \times 9.8}{1.0} = 9.8 \text{ m/s}$$

### Observações

1. **Efeito da força de arraste**: A força de arraste proporcional à velocidade ($F_{arraste} = \alpha v$) compensa gradualmente a força peso, levando o objeto a uma velocidade limite.

2. **Convergência**: O objeto aproxima-se assintoticamente da velocidade terminal, nunca a atingindo completamente em tempo finito.

3. **Precisão numérica**: O método das diferenças finitas (MDF) apresenta boa concordância com a solução analítica, especialmente para passos de tempo menores.

4. **Estabilidade**: A relação iterativa é estável para os valores de $\delta t$ utilizados, mantendo a convergência para a solução correta.
